In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from math import pi
from matplotlib.lines import Line2D
import tomllib
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import r2_score
import random
import snoop
import warnings
from PIL import Image
import matplotlib as mpl

In [ ]:
meteo = pd.read_excel("meteo.xlsx")
ugv = pd.read_excel("ugv.xlsx")
meteo["id_s"] = np.array(meteo["id_s"], dtype=str)
ugv["id_s"] = np.array(ugv["id_s"], dtype=str)
print(len(meteo).len(ugv))

In [2]:
# @snoop
def loocv(x, y, model):
    warnings.filterwarnings("ignore")
    """х - массив точек.
    у - массив значений.
    возвращаяет R2 LOOCV модели"""
    pred = []
    loo = LeaveOneOut()
    if type(x) is not np.array:
        x = np.array(x)
    if type(y) is not np.array:
        y = np.array(y)
    for i, (train_index, test_index) in enumerate(loo.split(x)):
        model.fit(x[train_index], y[train_index])
        res = model.predict(np.array(x[test_index]))
        pred.append(float(res))
    return r2_score(y, pred)

In [3]:
def gradient(x, y):
    """x.у - массивы с координатами по x и y осям.\n
    возвращает максимальный градиент функции"""
    q, t = [], []
    for s in np.argsort(x):
        q.append(x[s])
        t.append(y[s])
    grad = []
    for i in range(len(q) - 1):
        if q[i + 1] - q[i] == 0:
            grad.append(0)
        else:
            grad.append((t[i + 1] - t[i]) / (q[i + 1] - q[i]))
    return np.mean(np.abs(grad))

In [4]:
c2 = [
    [2, 0.015],
    [2, 0.0198],
    [2, 0.024],
    [2, 0.03],
    [2, 0.036],
    [2, 0.0396],
    [2, 0.045],
    [2, 0.048],
    [2, 0.051],
    [3, 0.033],
    [2.55, 0.0465],
    [2.7, 0.0105],
    [2.85, 0.024],
    [2.4, 0.0285],
    [2.25, 0.042],
    [1.8, 0.0375],
    [1.87, 0.051],
    [2.13, 0.051],
    [1.65, 0.051],
    [3, 0.051],
]

In [4]:
def custom_cmap(c1, c2, n):
    """Return object cmap.
    :c1: array [r,g,b,alpha] 0-1 or 0-255. or color name from matplotlib
    :c2: array [r,g,b,alpha] 0-1 or 0-255. or color name from matplotlib
    :n: number of gradation
    """
    if type(c1) != str:
        c1 = np.array(c1)
        c2 = np.array(c2)
        if np.logical_or(c1 > 1, c2 > 1).any():
            c1 = c1 / 255
            c2 = c2 / 255
    else:
        c1 = mpl.colors.to_rgb(c1)
        c2 = mpl.colors.to_rgb(c2)
    q = np.array(
        [
            np.linspace(c1[0], c2[0], n),
            np.linspace(c1[1], c2[1], n),
            np.linspace(c1[2], c2[2], n),
            np.linspace(c1[3], c2[3], n),
        ]
    )
    colorMap = mpl.colors.ListedColormap(q.T)
    return colorMap

In [113]:
temp = {}
coordinates = {}
mul, shift = 8, 4
for i in range(50):
    x, y = round(random.random() * mul - shift, 3), round(
        random.random() * mul - shift, 3
    )
    coordinates[str(i)] = [x, y]
    peaks = [
        (3, 3, 4),  # пик высотой 4
        (-4, -2, 3),  # пик высотой 3
        (0, -5, 5),  # пик высотой 5 (самый высокий)
    ]

    for px, py, height in peaks:
        sigma = 2.0
        distance = np.sqrt((x - px) ** 2 + (y - py) ** 2)
        peak_contribution = height * np.exp(-(distance**2) / (2 * sigma**2))
        z += peak_contribution
    temp[str(i)] = z

    # temp[str(i)]=(2 * np.exp(-((x - 3)**2 + (y - 3)**2) / 4) +      # Положительный пик в (3, 3)
    #      1.5 * np.exp(-((x + 4)**2 + (y - 2)**2) / 3) -     # Положительный пик в (-4, 2)
    #      1.8 * np.exp(-((x + 2)**2 + (y + 3)**2) / 2) -     # Отрицательный пик (впадина) в (-2, -3)
    #      1.2 * np.exp(-((x - 5)**2 + (y + 4)**2) / 2.5) +   # Отрицательный пик в (5, -4)
    #      0.5 * np.sin(0.5 * x) * np.cos(0.5 * y))           # Добавляем небольшие колебания

In [156]:
def generate_surface(
    nums, x_range=(-6, 6), y_range=(-6, 6), grid_resolution=1000, seed=None
):
    """
    Упрощенная функция: генерирует поверхность и возвращает только 50 точек.

    Возвращает:
    - points: массив формы (50, 3) с координатами [x, y, z]
    """
    if seed is not None:
        np.random.seed(seed)

    # Создаем плотную сетку для генерации поверхности
    x = np.linspace(x_range[0], x_range[1], grid_resolution)
    y = np.linspace(y_range[0], y_range[1], grid_resolution)
    X, Y = np.meshgrid(x, y)

    # Базовая поверхность
    Z = np.full_like(X, 0)

    # Три максимума
    peaks = [(3, 3, 4), (-5, -5, 5), (0, 0, 2)]
    for px, py, height in peaks:
        sigma = 2.0
        distance_squared = (X - px) ** 2 + (Y - py) ** 2
        peak = height * np.exp(-distance_squared / (2 * sigma**2))
        Z += peak

    # Выбираем 50 случайных точек
    all_points = np.column_stack([X.ravel(), Y.ravel(), Z.ravel()])
    indices = np.random.choice(len(all_points), nums, replace=False)
    selected_points = all_points[indices]

    return selected_points


In [153]:
def create_map(
    data: dict,
    coordinates: dict,
    resolution=100,
    model=ExtraTreesRegressor(),
    validation=False,
    normalize=True,
    gap=10,
):

    """

    Создает интерполированную карту значений на основе заданных данных и координат.


    Функция использует машинное обучение для интерполяции значений между известными точками

    и генерирует регулярную сетку значений, которая может быть использована для визуализации

    карты изолиний, тепловой карты или других типов пространственного анализа.


    Параметры:
    -----------

    data : dict

        Словарь, где ключи - идентификаторы точек, значения - числовые значения в этих точках

    coordinates : dict

        Словарь, где ключи - те же идентификаторы точек, значения - кортежи координат (x, y)

    resolution : int, optional, default=100

        Разрешение выходной сетки (количество точек по каждой оси)

    model : sklearn estimator, optional, default=ExtraTreesRegressor()

        Модель машинного обучения для интерполяции значений

    normalize : bool, optional, default=True

        Флаг нормализации координат перед обучением модели

    gap : int, optional, default=10

        Процент расширения границ карты относительно крайних точек (для отступа)


    Возвращает:
    -----------

    aa : numpy.ndarray

        2D массив координат X сетки (meshgrid)

    bb : numpy.ndarray

        2D массив координат Y сетки (meshgrid)

    z_map : numpy.ndarray

        2D массив интерполированных значений Z, соответствующих координатам сетки


    Пример использования:
    --------------------

    >>> data = {'point1': 10, 'point2': 20, 'point3': 15}

    >>> coordinates = {'point1': (0, 0), 'point2': (10, 10), 'point3': (5, 3)}

    >>> xx, yy, zz = create_map(data, coordinates, resolution=50)

    """

    X = np.array(
        list(coordinates.values())
    )  # X- массив координат [[x1.y1].[x2.y2]...]        format = "%.0f"


    x, y = X.T[0], X.T[1]  # координаты X и Y отдельно


    bounds_x = [min(x), max(x)]

    bounds_y = [min(y), max(y)]


    delta_x = (bounds_x[1] - bounds_x[0]) * gap / 100

    delta_y = (bounds_y[1] - bounds_y[0]) * gap / 100

    resolution = int(resolution)

    aa, bb = np.meshgrid(

        np.linspace(bounds_x[0] - delta_x, bounds_x[1] + delta_x, resolution),

        np.linspace(bounds_y[0] - delta_y, bounds_y[1] + delta_y, resolution),
    )

    xx = np.vstack([aa.ravel(), bb.ravel()]).T


    if normalize:

        scaler = StandardScaler()

        learn_X = scaler.fit_transform(np.array(list(coordinates.values())))
        pred_xx = scaler.fit_transform(xx)

    else:
        learn_X = X
        pred_xx = xx

    learn_Z = np.array(list(data.values())).reshape(-1, 1)  # массив значений

    model.fit(learn_X, learn_Z)

    z_map = model.predict(pred_xx).reshape(resolution, resolution)

    if validation:

        r2 = loocv(X, learn_Z, model)

        print("R2-score: ", r2)

    return aa, bb, z_map

In [161]:
from pyfitit import ML


peaks = generate_surface(50, (-10, 10), (-10, 10))


coordinates, temp = {}, {}
for i in range(len(peaks)):
    coordinates[str(i)] = peaks[i][0:2]
    temp[str(i)] = peaks[i][2]
    
x, y, z = create_map(
    temp,
    coordinates,
    model=ML.RBF(),
    resolution=200,
    normalize=False,
    validation=True,
    gap=10,
)

R2-score:  0.9322675100683581


In [162]:
# @snoop
def plot_map(
    xx, yy, zz, points={}, points_value={}, img=None, levels=10, cmap="custom"
):
    plt.figure(figsize=(12, 10))
    if img != None:
        alpha_map = 0.5
        plt.imshow(img, extent=[np.min(xx), np.max(xx), np.min(yy), np.max(yy)])
    else:
        alpha_map = 1

    if cmap == "custom":
        cmap = custom_cmap([0, 1, 0, 0.6], [1, 0, 0, 0.6], levels)
    plotparam = {"levels": levels, "cmap": cmap}
    plotparam2 = {"levels": levels, "colors": "black", "linewidths": 0.5}

    countr = plt.contourf(xx, yy, zz, **plotparam)
    plt.contour(xx, yy, zz, **plotparam2)
    plt.colorbar(countr, shrink=0.75)

    lev = countr.levels
    cm = plt.get_cmap(cmap)
    colors = cm(np.linspace(0, 1, len(lev) + 1))
    if len(points) != 0:
        poi = np.array(list(points.values()))
        names = list(points.keys())
        x, y = poi.T[0], poi.T[1]
        poi_v = list(points_value.values())
        size = 500
        color_index = np.searchsorted(lev, poi_v, side="left")
        color_index[color_index == max(color_index)] = max(color_index) - 1
        for i in range(len(names)):
            plt.scatter(
                x[i], y[i], c=colors[color_index[i]], s=size, edgecolors="black"
            )
            plt.text(x[i], y[i], names[i], ha="center", va="center")

    plt.gca().set_aspect("equal")  #'
    plt.tight_layout()
    plt.savefig("aaa.png")
    plt.close()


img = Image.open("map.png")
#
plot_map(x, y, z, img=img, points=coordinates, points_value=temp, levels=7)
# for i in range(10):
#     print(i/10)
#     plot_map(x,y,z,i/10,img=img,points=coordinates,points_value=temp)